# Cyberbullying detection
## Modeling
<hr>

In [42]:
#Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import re
from collections import defaultdict

from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Embedding



%matplotlib inline

<hr>
read the dataset
<hr>

In [9]:
df_cbd = pd.read_csv('../datasets/twitter_parsed_dataset_post_cleaning.csv')

In [5]:
df_cbd.head(20)

,index,id,text,annotation,oh_label
0,5.749487e+17,5.749487e+17,halalflaws biebervalue greenlinerzjm read cont...,none,0.0
1,5.719179e+17,5.719179e+17,shreyabafna3 idiots claim people tried stop be...,none,0.0
2,3.902558e+17,3.902558e+17,mooseoftorment call sexist go auto place id ra...,sexism,1.0
3,5.682089e+17,5.682089e+17,g0ssipsquirrelx wrong isis follows example moh...,racism,1.0
4,5.755963e+17,5.755963e+17,mkr,none,0.0
5,5.611848e+17,5.611848e+17,trobinsonnewera saudi preacher raped tortured ...,none,0.0
6,4.365631e+17,4.365631e+17,millhouse66 maureenjs nooo sexist women bad dr...,sexism,1.0
7,5.759623e+17,5.759623e+17,going make pancakesdont hve strawberries hve b...,none,0.0
8,5.725815e+17,5.725815e+17,ahtweet freebsdgirl dare feelings fantastic wa...,none,0.0
9,5.004564e+17,5.004564e+17,newmanzaa theres something wrong girl wins way...,sexism,1.0


Now I have to create the X and y arrays

In [10]:
X = df_cbd['text']
y = df_cbd['oh_label']

In [15]:
X.fillna(" ", axis=0, inplace=True)

In [21]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X.values)
sequences = tokenizer.texts_to_sequences(X.values)

In [22]:
vocab_size=len(tokenizer.word_index) + 1
print (f'Size of vocabulary : {vocab_size}')

Size of vocabulary : 23352


In [19]:
print(tokenizer.word_index)

{'mkr': 1, 'im': 2, 'sexist': 3, 'kat': 4, 'like': 5, 'islam': 6, 'dont': 7, 'women': 8, 'people': 9, 'amp': 10, 'get': 11, 'one': 12, 'think': 13, 'andre': 14, 'would': 15, 'isis': 16, 'girls': 17, 'know': 18, 'muslims': 19, 'cant': 20, 'go': 21, 'want': 22, 'time': 23, 'good': 24, 'really': 25, 'going': 26, 'see': 27, 'make': 28, 'men': 29, 'muslim': 30, 'even': 31, 'mkr2015': 32, 'oh': 33, 'never': 34, 'thats': 35, 'need': 36, 'call': 37, 'still': 38, 'way': 39, 'right': 40, 'well': 41, 'say': 42, 'mykitchenrules': 43, 'maxblumenthal': 44, 'hate': 45, 'youre': 46, 'ive': 47, 'woman': 48, 'much': 49, 'back': 50, 'fuck': 51, 'female': 52, 'got': 53, 'shit': 54, 'doesnt': 55, 'mohammed': 56, 'us': 57, 'notsexist': 58, 'didnt': 59, 'ever': 60, 'love': 61, 'stop': 62, 'daesh': 63, 'every': 64, 'many': 65, 'look': 66, 'quran': 67, 'someone': 68, 'twitter': 69, 'thing': 70, 'another': 71, 'show': 72, 'freebsdgirl': 73, 'better': 74, 'man': 75, 'cook': 76, 'pretty': 77, 'lol': 78, 'yes': 79

In [33]:
max_length = max([len(s.split()) for s in X])

In [26]:
padded = pad_sequences(sequences, maxlen=max_length, padding='post')

Now the text is ready, I'm going to split X into train and test

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

### Model

<hr>

- Optimizer: Adam
- Loss function: binary cross-entropy (suited for binary classification problem)

Dropout layers are used to solve the problem of overfitting and bring generalization into the model. As a result, in hidden layers, it’s best to keep the dropout value near 0.5.

In [39]:
X_train.shape[1]

(11793,)

In [46]:
cb_model = Sequential()

cb_model.add(Embedding(vocab_size, 100, input_length=max_length))
cb_model.add(Flatten(input_shape=(X_train.shape,)))
cb_model.add(Dense(45, activation='relu'))
cb_model.add(Dropout(0.5))  
cb_model.add(Dense(1, activation='sigmoid'))
    

In [47]:
cb_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
cb_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 24, 100)           2335200   
                                                                 
 flatten (Flatten)           (None, 2400)              0         
                                                                 
 dense (Dense)               (None, 45)                108045    
                                                                 
 dropout (Dropout)           (None, 45)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 46        
                                                                 
Total params: 2443291 (9.32 MB)
Trainable params: 2443291 (9.32 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [50]:
cb_model.fit(X_train, y_train.values, epochs=10, verbose=2, batch_size=128)

Epoch 1/10


ValueError: in user code:

    File "/home/hmorales/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/home/hmorales/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/hmorales/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/home/hmorales/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/home/hmorales/.local/lib/python3.8/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/hmorales/.local/lib/python3.8/site-packages/keras/src/engine/input_spec.py", line 280, in assert_input